# Basic Cellular Automata

An exercise in the study of image convolution kernels

[Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from matplotlib.animation import FuncAnimation

def initialize_grid(size):
    # Initialize the grid
    return np.random.choice([0, 1], size=size*size, p=[0.9, 0.1]).reshape(size, size) #P(D,A)

def update_grid(grid):
    # Update the grid based on the GOL rules
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])
    neighbors = convolve2d(grid, kernel, mode='same', boundary='wrap')
    # >3 neighbors to come to life; >2 neighbors to stay alive
    return (neighbors == 3) | ((grid == 1) & (neighbors == 2))

def update_plot(frame_num, img, grid):
    # Update the plot for animation
    new_grid = update_grid(grid)
    img.set_data(new_grid)
    grid[:] = new_grid 
    return img,

# Set the size of the grid and the number of frames for the animation
size = 100
num_frames = 200 

# Initialize the grid
grid = initialize_grid(size)

# Setup the plot
fig, ax = plt.subplots()
img = ax.imshow(grid, cmap='gray')
ax.axis('off')

# Create and save the animation
ani = FuncAnimation(fig, update_plot, frames=num_frames, fargs=(img, grid), blit=True)
ani.save('game_of_life.mp4', writer='ffmpeg', fps=10)

plt.close()


## Kernel Definition

A **kernel** in image processing (and, by extension, in grid-based calculations like cellular automata) is a small matrix used to apply operations such as edge detection, blurring, sharpening, and more. In the context of the Game of Life, the kernel is used to count the number of alive neighbors each cell has.

Here’s the kernel used in the `update_grid` function:

$$
\text{kernel} = \begin{bmatrix}
1 & 1 & 1 \\
1 & 0 & 1 \\
1 & 1 & 1 
\end{bmatrix} 
$$

This kernel is designed to be used with the convolution operation. The center cell (where the kernel has a 0) is the cell being updated, and the surrounding cells (where the kernel has 1s) are its neighbors.

**Convolution Operation**:

The `convolve2d` function is used to apply this kernel to the grid. It overlays the kernel on top of each cell in the grid (the center of the kernel aligns with the current cell), multiplies the overlapping kernel and grid values, and sums the result. Since the kernel has 1s in positions corresponding to the neighbors and 0 at the center, the result of this convolution at each position is the sum of the alive neighbors for that cell.

**Applying the Game of Life Rules**:

After calculating the number of alive neighbors for each cell, the next grid state is determined by applying the rules of the Game of Life:

1. Any live cell with two or three live neighbors survives.
2. Any dead cell with exactly three live neighbors becomes a live cell.
3. All other live cells die in the next generation. Similarly, all other dead cells stay dead.

The rules are implemented as follows:

$$
\text{return} \; (\text{neighbors} == 3) \; | \; ((\text{grid} == 1) \& (\text{neighbors} == 2))
$$

- $(\text{neighbors} == 3)$ checks which cells have exactly three neighbors. These cells will be alive in the next generation regardless of their current state (rule of birth by reproduction or survival).
- $((\text{grid} == 1) \& (\text{neighbors} == 2))$ checks which cells are currently alive ($\text{grid} == 1$) and have exactly two neighbors. These cells will continue to be alive (survival).

The result of these conditions is a boolean grid where `True` represents an alive cell and `False` represents a dead cell. This boolean grid becomes the new state of the grid for the next iteration.